In [21]:
import numpy as np
import pickle
import torch


def xavier_init(size, gain = 1.0):
    """
    Xavier initialization of network weights.

    Arguments:
        - size {tuple} -- size of the network to initialise.
        - gain {float} -- gain for the Xavier initialisation.

    Returns:
        {np.ndarray} -- values of the weights.
    """
    low = -gain * np.sqrt(6.0 / np.sum(size))
    high = gain * np.sqrt(6.0 / np.sum(size))
    return np.random.uniform(low=low, high=high, size=size)


class Layer:
    """
    Abstract layer class.
    """

    def __init__(self, *args, **kwargs):
        raise NotImplementedError()

    def forward(self, *args, **kwargs):
        raise NotImplementedError()

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

    def backward(self, *args, **kwargs):
        raise NotImplementedError()

    def update_params(self, *args, **kwargs):
        pass


class MSELossLayer(Layer):
    """
    MSELossLayer: Computes mean-squared error between y_pred and y_target.
    """

    def __init__(self):
        self._cache_current = None

    @staticmethod
    def _mse(y_pred, y_target):
        return np.mean((y_pred - y_target) ** 2)

    @staticmethod
    def _mse_grad(y_pred, y_target):
        return 2 * (y_pred - y_target) / len(y_pred)

    def forward(self, y_pred, y_target):
        self._cache_current = y_pred, y_target
        return self._mse(y_pred, y_target)

    def backward(self):
        return self._mse_grad(*self._cache_current)


class CrossEntropyLossLayer(Layer):
    """
    CrossEntropyLossLayer: Computes the softmax followed by the negative 
    log-likelihood loss.
    """

    def __init__(self):
        self._cache_current = None

    @staticmethod
    def softmax(x):
        numer = np.exp(x - x.max(axis=1, keepdims=True))
        denom = numer.sum(axis=1, keepdims=True)
        return numer / denom

    def forward(self, inputs, y_target):
        assert len(inputs) == len(y_target)
        n_obs = len(y_target)
        probs = self.softmax(inputs)
        self._cache_current = y_target, probs

        out = -1 / n_obs * np.sum(y_target * np.log(probs))
        return out

    def backward(self):
        y_target, probs = self._cache_current
        n_obs = len(y_target)
        return -1 / n_obs * (y_target - probs)


class SigmoidLayer(Layer):
    """
    SigmoidLayer: Applies sigmoid function elementwise.
    """

    def __init__(self):
        """ 
        Constructor of the Sigmoid layer.
        """
        self._cache_current = None

    def forward(self, x):
        """ 
        Performs forward pass through the Sigmoid layer.

        Logs information needed to compute gradient at a later stage in
        `_cache_current`.

        Arguments:
            x {np.ndarray} -- Input array of shape (batch_size, n_in).

        Returns:
            {np.ndarray} -- Output array of shape (batch_size, n_out)
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        #store the input in the cache to later use in backward pass
        self._cache_current = x
        #return x passed through the sigmoid function
        return 1/(1+np.exp(-x))

        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def backward(self, grad_z):
        """
        Given `grad_z`, the gradient of some scalar (e.g. loss) with respect to
        the output of this layer, performs back pass through the layer (i.e.
        computes gradients of loss with respect to parameters of layer and
        inputs of layer).

        Arguments:
            grad_z {np.ndarray} -- Gradient array of shape (batch_size, n_out).

        Returns:
            {np.ndarray} -- Array containing gradient with repect to layer
                input, of shape (batch_size, n_in).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        #retrieve input from the cache
        x = self._cache_current
        #compute sigmoid derivative of output of previous linear layer (input of activation function)
        sigmoid_derivative = np.exp(-x)/(1+np.exp(-x))**2
        
        #check whether same shape
        assert grad_z.shape == sigmoid_derivative.shape
        
        #do element wise multiplication of gradient wrt to output of activation function and sigmoid derivative 
        return np.multiply(grad_z, sigmoid_derivative)

        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################


class ReluLayer(Layer):
    """
    ReluLayer: Applies Relu function elementwise.
    """

    def __init__(self):
        """
        Constructor of the Relu layer.
        """
        self._cache_current = None

    def forward(self, x):
        """ 
        Performs forward pass through the Relu layer.

        Logs information needed to compute gradient at a later stage in
        `_cache_current`.

        Arguments:
            x {np.ndarray} -- Input array of shape (batch_size, n_in).

        Returns:
            {np.ndarray} -- Output array of shape (batch_size, n_out)
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        #store the input in the cache to later use in backward pass
        self._cache_current = x
        #return x passed through the relu function
        return np.maximum(0,x)

        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def backward(self, grad_z):
        """
        Given `grad_z`, the gradient of some scalar (e.g. loss) with respect to
        the output of this layer, performs back pass through the layer (i.e.
        computes gradients of loss with respect to parameters of layer and
        inputs of layer).

        Arguments:
            grad_z {np.ndarray} -- Gradient array of shape (batch_size, n_out).

        Returns:
            {np.ndarray} -- Array containing gradient with repect to layer
                input, of shape (batch_size, n_in).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        #retrieve input from the cache
        x = self._cache_current
        #compute relu derivative of output of previous linear layer (input of activation function)
        relu_derivative = (x > 0) * 1
        
        #check whether same shape
        assert grad_z.shape == relu_derivative.shape
        
        #do element wise multiplication of gradient wrt to output of activation function and relu derivative 
        return np.multiply(grad_z, relu_derivative)

        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################


class LinearLayer(Layer):
    """
    LinearLayer: Performs affine transformation of input.
    """

    def __init__(self, n_in, n_out):
        """
        Constructor of the linear layer.

        Arguments:
            - n_in {int} -- Number (or dimension) of inputs.
            - n_out {int} -- Number (or dimension) of outputs.
        """
        self.n_in = n_in
        self.n_out = n_out

        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        # initialize weights of linear layer with Xavier Glorot initialization (gain of 1)
        self._W = xavier_init((self.n_in, self.n_out), 1.0)
        # initialize biases with zeros, because we don’t want the neurons to start out with a bias
        self._b = np.zeros((1,self.n_out))

        self._cache_current = None
        self._grad_W_current = None
        self._grad_b_current = None

        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def forward(self, x):
        """
        Performs forward pass through the layer (i.e. returns Wx + b).

        Logs information needed to compute gradient at a later stage in
        `_cache_current`.

        Arguments:
            x {np.ndarray} -- Input array of shape (batch_size, n_in).

        Returns:
            {np.ndarray} -- Output array of shape (batch_size, n_out)
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        #retrieve input from the cache
        self._cache_current = x
        
        #check whether same shape
        assert (x.shape[1] == self._W.shape[0])
        
        # calculate output of linear layer and return
        z = x @ self._W + np.repeat(self._b,x.shape[0],axis=0)
        
        return z
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def backward(self, grad_z):
        """
        Given `grad_z`, the gradient of some scalar (e.g. loss) with respect to
        the output of this layer, performs back pass through the layer (i.e.
        computes gradients of loss with respect to parameters of layer and
        inputs of layer).

        Arguments:
            grad_z {np.ndarray} -- Gradient array of shape (batch_size, n_out).

        Returns:
            {np.ndarray} -- Array containing gradient with respect to layer
                input, of shape (batch_size, n_in).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        # check whether same shape and multiplications can be performed
        
        assert (np.transpose(self._cache_current).shape[1] == grad_z.shape[0]) & (np.ones((1,self._cache_current.shape[0])).shape[1] == grad_z.shape[0]) & (grad_z.shape[1] == np.transpose(self._W).shape[0])
        
        # calculate loss gradient wrt to weights: input_transpose * grad_z
        self._grad_W_current = np.transpose(self._cache_current) @ grad_z
        # calculate loss gradient wrt to bias: column vector of ones * grad_z
        self._grad_b_current = np.ones((1,self._cache_current.shape[0])) @ grad_z
        
        #return gradient with respect to the inputs of the layer: grad_z * w_transpose
        return grad_z @ np.transpose(self._W)

        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def update_params(self, learning_rate):
        """
        Performs one step of gradient descent with given learning rate on the
        layer's parameters using currently stored gradients.

        Arguments:
            learning_rate {float} -- Learning rate of update step.
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        #update weights and biases 
        self._W += - learning_rate * self._grad_W_current
        self._b += - learning_rate * self._grad_b_current

        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################


class MultiLayerNetwork(object):
    """
    MultiLayerNetwork: A network consisting of stacked linear layers and
    activation functions.
    """

    def __init__(self, input_dim, neurons, activations):
        """
        Constructor of the multi layer network.

        Arguments:
            - input_dim {int} -- Number of features in the input (excluding 
                the batch dimension).
            - neurons {list} -- Number of neurons in each linear layer 
                represented as a list. The length of the list determines the 
                number of linear layers.
            - activations {list} -- List of the activation functions to apply 
                to the output of each linear layer.
        """
        self.input_dim = input_dim
        self.neurons = neurons
        self.activations = activations

        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        #stack all layers in one ndarray 
        layers = np.ndarray((len(self.neurons)*2),dtype=np.object)
        n_in = self.input_dim
        for i in range(len(self.neurons)):
            layers[2*i] = LinearLayer(n_in,self.neurons[i])
            n_in = self.neurons[i]
            if (self.activations[i] == 'relu'):
                layers[2*i+1] = ReluLayer()
            elif (self.activations[i] == 'sigmoid'):
                layers[2*i+1] = SigmoidLayer()
            else:
                layers[2*i+1] = 'identity'
            
        self._layers = layers
                
            
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def forward(self, x):
        """
        Performs forward pass through the network.

        Arguments:
            x {np.ndarray} -- Input array of shape (batch_size, input_dim).

        Returns:
            {np.ndarray} -- Output array of shape (batch_size,
                #_neurons_in_final_layer)
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        #pass through all layers
        z_temp = x
        for i in range(len(self._layers)):
            #check whether identity layer and no activation function should be called
            if not self._layers[i] == 'identity':
                z_temp = self._layers[i].forward(z_temp)
        return z_temp 

        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def __call__(self, x):
        return self.forward(x)

    def backward(self, grad_z):
        """
        Performs backward pass through the network.

        Arguments:
            grad_z {np.ndarray} -- Gradient array of shape (1,
                #_neurons_in_final_layer).

        Returns:
            {np.ndarray} -- Array containing gradient with repect to layer
                input, of shape (batch_size, input_dim).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        #backwards pass through all layers
        grad_z_temp = grad_z
        for i in range(len(self._layers)-1,-1,-1):
            #check whether identity layer and no activation function should be called
            if not self._layers[i] == 'identity':
                grad_z_temp = self._layers[i].backward(grad_z_temp)
        return grad_z_temp
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def update_params(self, learning_rate):
        """
        Performs one step of gradient descent with given learning rate on the
        parameters of all layers using currently stored gradients.

        Arguments:
            learning_rate {float} -- Learning rate of update step.
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        #update all weights and biases in the individual layers
        for i in range(len(self._layers)):
            #check that only linear layers are called
            if isinstance(self._layers[i], LinearLayer):
                self._layers[i].update_params(learning_rate)
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################


def save_network(network, fpath):
    """
    Utility function to pickle `network` at file path `fpath`.
    """
    with open(fpath, "wb") as f:
        pickle.dump(network, f)


def load_network(fpath):
    """
    Utility function to load network found at file path `fpath`.
    """
    with open(fpath, "rb") as f:
        network = pickle.load(f)
    return network


class Trainer(object):
    """
    Trainer: Object that manages the training of a neural network.
    """

    def __init__(
        self,
        network,
        batch_size,
        nb_epoch,
        learning_rate,
        loss_fun,
        shuffle_flag,
        use_torch
    ):
        """
        Constructor of the Trainer.

        Arguments:
            - network {MultiLayerNetwork} -- MultiLayerNetwork to be trained.
            - batch_size {int} -- Training batch size.
            - nb_epoch {int} -- Number of training epochs.
            - learning_rate {float} -- SGD learning rate to be used in training.
            - loss_fun {str} -- Loss function to be used. Possible values: mse,
                bce.
            - shuffle_flag {bool} -- If True, training data is shuffled before
                training.
        """
        self.network = network
        self.batch_size = batch_size
        self.nb_epoch = nb_epoch
        self.learning_rate = learning_rate
        self.loss_fun = loss_fun
        self.shuffle_flag = shuffle_flag
        self.use_torch = use_torch
        if self.use_torch:
            self.optimizer = torch.optim.SGD(self.network.parameters(), lr=self.learning_rate)

        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        if loss_fun == 'cross_entropy':
            self._loss_layer = CrossEntropyLossLayer()
        elif loss_fun == 'mse':
            self._loss_layer = MSELossLayer()
        #self._loss_layer = CrossEntropyLossLayer() if loss_fun == 'cross_entropy' else self._loss_layer = MSELossLayer()
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    @staticmethod
    def shuffle(input_dataset, target_dataset):
        """
        Returns shuffled versions of the inputs.

        Arguments:
            - input_dataset {np.ndarray} -- Array of input features, of shape
                (#_data_points, n_features).
            - target_dataset {np.ndarray} -- Array of corresponding targets, of
                shape (#_data_points, #output_neurons).

        Returns: 
            - {np.ndarray} -- shuffled inputs.
            - {np.ndarray} -- shuffled_targets.
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        #randomly shuffle indices and use shuffled indices to get shuffled dataset (both input and target)
        idx_list = np.arange(len(input_dataset))
        np.random.shuffle(idx_list)
        shuffled_inputs  = input_dataset[idx_list]
        shuffled_targets  = target_dataset[idx_list]
        return shuffled_inputs, shuffled_targets
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def train(self, input_dataset, target_dataset):
        """
        Main training loop. Performs the following steps `nb_epoch` times:
            - Shuffles the input data (if `shuffle` is True)
            - Splits the dataset into batches of size `batch_size`.
            - For each batch:
                - Performs forward pass through the network given the current
                batch of inputs.
                - Computes loss.
                - Performs backward pass to compute gradients of loss with
                respect to parameters of network.
                - Performs one step of gradient descent on the network
                parameters.

        Arguments:
            - input_dataset {np.ndarray} -- Array of input features, of shape
                (#_training_data_points, n_features).
            - target_dataset {np.ndarray} -- Array of corresponding targets, of
                shape (#_training_data_points, #output_neurons).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        if self.use_torch:
            criterion = torch.nn.CrossEntropyLoss()# cross entropy loss
        #loop over number of epochs
        for i in range(self.nb_epoch):
            #check whether data should be shuffled
            if self.shuffle_flag == True:
                input_dataset, target_dataset = self.shuffle(input_dataset, target_dataset)
            #Splits the dataset into batches of size batch_size
            input_batches = [input_dataset[i:i + self.batch_size] for i in range(0, len(input_dataset), self.batch_size)]  
            target_batches = [target_dataset[i:i + self.batch_size] for i in range(0, len(target_dataset), self.batch_size)] 
            
            #loop over batches
            if self.use_torch == False:
                for i in range(len(input_batches)):
                    y_pred = self.network(input_batches[i])
                    self._loss_layer.forward(y_pred, target_batches[i])
                    grad_loss = self._loss_layer.backward()
                    self.network.backward(grad_loss)
                    self.network.update_params(self.learning_rate)
            else:
                for i in range(len(input_batches)):
                    self.optimizer.zero_grad()
                    y_pred = self.network.forward(torch.tensor(input_batches[i], dtype=torch.float32))
                    labels = torch.tensor(target_batches[i], dtype=torch.float32).argmax(1)
                    loss = criterion(y_pred, labels)
                    loss.backward()
                    self.optimizer.step()
                    

        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def eval_loss(self, input_dataset, target_dataset):
        """
        Function that evaluate the loss function for given data.

        Arguments:
            - input_dataset {np.ndarray} -- Array of input features, of shape
                (#_evaluation_data_points, n_features).
            - target_dataset {np.ndarray} -- Array of corresponding targets, of
                shape (#_evaluation_data_points, #output_neurons).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        #get current target prediction and return calculated loss (with target data)
        if self.use_torch == False:
            y_pred = self.network(input_dataset)
            return self._loss_layer.forward(y_pred, target_dataset)
        else:
            y_pred = self.network.forward(torch.tensor(input_dataset, dtype=torch.float32))
            criterion = torch.nn.CrossEntropyLoss()
            labels = torch.tensor(target_dataset, dtype=torch.float32).argmax(1)
            return criterion(y_pred, labels)
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################


class Preprocessor(object):
    """
    Preprocessor: Object used to apply "preprocessing" operation to datasets.
    The object can also be used to revert the changes.
    """

    def __init__(self, data):
        """
        Initializes the Preprocessor according to the provided dataset.
        (Does not modify the dataset.)

        Arguments:
            data {np.ndarray} dataset used to determine the parameters for
            the normalization.
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        #find maximum and minimum in data and set a and b according to range [0,1]
        self.maximum = np.amax(data, axis = 0)
        self.minimum = np.amin(data, axis = 0)
        self.a = np.ones(data.shape[1])
        self.b = np.zeros(data.shape[1])

        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def apply(self, data):
        """
        Apply the pre-processing operations to the provided dataset.

        Arguments:
            data {np.ndarray} dataset to be normalized.

        Returns:
            {np.ndarray} normalized dataset.
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        #perform min max normalization : Scaling the smallest value to a and largest value to b
        normalized_data = self.a + (data - self.minimum) * (self.b-self.a) / (self.maximum - self.minimum)
        return normalized_data
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def revert(self, data):
        """
        Revert the pre-processing operations to retreive the original dataset.

        Arguments:
            data {np.ndarray} dataset for which to revert normalization.

        Returns:
            {np.ndarray} reverted dataset.
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        #revert min max normalization : Scaling a to smallest value and b to largest value
        reverted_data = (data - self.a) * (self.maximum - self.minimum) / (self.b-self.a) + self.minimum 
        return reverted_data

        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

        
######################## Testing with torch #############################      



# The Network class inherits the torch.nn.Module class, which represents a neural network.
class Network(torch.nn.Module):

    # The class initialisation function. This takes as arguments the dimension of the network's input (i.e. the dimension of the state), and the dimension of the network's output (i.e. the dimension of the action).
    def __init__(self):
        # Call the initialisation function of the parent class.
        super(Network, self).__init__()
        self.layer_1 = torch.nn.Linear(in_features=4, out_features=16)    
        self.output_layer = torch.nn.Linear(in_features=16, out_features=3)
        self.softmax = torch.nn.Softmax(dim=1)

    # Function which sends some input data through the network and returns the network's output. In this example, a ReLU activation function is used for both hidden layers, but the output layer has no activation function (it is just a linear layer).
    def forward(self, input):
        layer_1_output = torch.nn.functional.relu(self.layer_1(input))
        #layer_2_output = torch.nn.functional.relu(self.layer_2(layer_1_output))
        #output = self.output_layer(layer_1_output)
        output = self.softmax(self.output_layer(layer_1_output))
        return output
        
        
        
        


def example_main():
    input_dim = 4
    neurons = [16, 3]
    activations = ["relu", "identity"]
    net = MultiLayerNetwork(input_dim, neurons, activations)

    net_torch = Network()
    
    dat = np.loadtxt("iris.dat")
    np.random.shuffle(dat)

    x = dat[:, :4]
    y = dat[:, 4:]

    split_idx = int(0.8 * len(x))

    x_train = x[:split_idx]
    y_train = y[:split_idx]
    x_val = x[split_idx:]
    y_val = y[split_idx:]

    prep_input = Preprocessor(x_train)

    x_train_pre = prep_input.apply(x_train)
    x_val_pre = prep_input.apply(x_val)

    trainer = Trainer(
        network=net,
        batch_size=8,
        nb_epoch=1000,
        learning_rate=0.01,
        loss_fun="cross_entropy",
        shuffle_flag=True,
        use_torch = False
    )
    
    
    trainer_torch = Trainer(
        network=net_torch,
        batch_size=8,
        nb_epoch=1000,
        learning_rate=0.01,
        loss_fun="cross_entropy",
        shuffle_flag=True,
        use_torch = True
    )

    trainer.train(x_train_pre, y_train)
    trainer_torch.train(x_train_pre, y_train)
    print("Train loss = ", trainer.eval_loss(x_train_pre, y_train))
    print("Validation loss = ", trainer.eval_loss(x_val_pre, y_val))
    print("Train torch loss = ", trainer_torch.eval_loss(x_train_pre, y_train))
    print("Validation torch loss = ", trainer_torch.eval_loss(x_val_pre, y_val))

    preds = net(x_val_pre).argmax(axis=1).squeeze()
    print("Predictions our network: {}".format(preds))
    preds_torch = net_torch.forward(torch.tensor(x_val_pre, dtype=torch.float32)).argmax(axis=1).squeeze().numpy()
    print("Predictions with torch: {}".format(preds_torch))
    targets = y_val.argmax(axis=1).squeeze()
    accuracy = (preds == targets).mean()
    accuracy_torch = (preds_torch == targets).mean()
    print("Validation accuracy: {}".format(accuracy))
    print("Validation torch accuracy: {}".format(accuracy_torch))


if __name__ == "__main__":
    example_main()



Train loss =  0.05880711382110921
Validation loss =  0.0772075373809868
Train torch loss =  tensor(0.6023, grad_fn=<NllLossBackward>)
Validation torch loss =  tensor(0.6159, grad_fn=<NllLossBackward>)
Predictions our network: [1 1 0 1 0 1 1 0 0 1 0 0 2 1 2 0 1 2 0 1 2 0 2 1 1 1 2 1 0 2]
Predictions with torch: [1 1 0 1 0 2 1 0 0 1 0 0 2 1 2 0 1 2 0 1 2 0 2 1 1 1 1 1 0 2]
Validation accuracy: 0.9666666666666667
Validation torch accuracy: 0.9666666666666667


In [ ]:
example_main()